In [ ]:
def intersection_over_union(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1]) 
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    interArea = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    iou = interArea / float(box1Area + box2Area - interArea)

    return iou

In [ ]:
#dikdörtgenler birbiri ile etkileşimde mi
def selectRelatedBox(coor, thresold):
    #seçili dikdörtgenlerin birbiri ile kesişimleri thresold değerinden büyük olanları al.
    selectCoor = {}
    start = 1
    for i in range(len(coor)):
        selectCoor[i]=[]
        for j in range(start,len(coor)):
            if (intersection_over_union(coor[i], coor[j]) > thresold):
                selectCoor[i].append(j)
        start+=1
        
    #birbiri ile alakalı dikdörtgenleri farklı dizilere ayırma
    split = []
    for i in range(len(selectCoor)):
        if (i not in split) and (selectCoor[i]):
            split.append(i)
            [split.append(x) for x in selectCoor[i] if x not in split]
            split.append("|")
    if not (split): return []
    del split[-1]
    

    splitted = {}
    start = 0
    splitted[start] = []
    for i in range(len(split)):
        if split[i] != "|":
            splitted[start].append(split[i])
        else:
            start+=1
            splitted[start]=[]
    return splitted

In [ ]:
import cv2, numpy

refPt = []
coordinates = []
area=[]
def click_and_crop(event, x, y, flags, param):

    global refPt, coordinates

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        coordinates.append((min(refPt[0][0],refPt[1][0]),min(refPt[0][1],refPt[1][1]),max(refPt[0][0],refPt[1][0]),max(refPt[0][1],refPt[1][1])))

        cv2.rectangle(img, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("image", img)


img = cv2.imread('1.jpg')
clone = img.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

while True:
    cv2.imshow("image", img)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("r"):
        img = clone.copy()
        coordinates = []

    elif key == ord("s"):
        break

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1

if len(coordinates) > 2:
    print(coordinates)
    coor = coordinates
    thresold=0.4

    #Aykırı değerlerin atılması
    arr_ = []
    arr = []
    for A in range(len(coor)):
        try:
            arr.append(((coor[A][3] - coor[A][1])/(coor[A][2] - coor[A][0]),coor[A]))    
            arr_.append(((coor[A][3] - coor[A][1])/(coor[A][2] - coor[A][0])))
        except:
            pass
    min_thresold, max_thresold = numpy.quantile(arr_,[0.15, 0.85])
    coor = [x[1] for x in arr if (x[0] < max_thresold) and (x[0] > min_thresold)]


    splittedCoor = selectRelatedBox(coor, thresold)
    if not splittedCoor: print("Doğruluk sağlanamadı")

    annatator=[]
    for i in range(len(splittedCoor)):
        newCoor = [coor[x] for x in splittedCoor[i]]

        boxes = numpy.array(newCoor)
        x1 = boxes[:,0]
        y1 = boxes[:,1]
        x2 = boxes[:,2]
        y2 = boxes[:,3]
        
        #İlişkili karelerin max sınırlarının alınıp iou değerlerine göre skor oluşturulması
        max_box = numpy.min(x1),numpy.min(y1),numpy.max(x2),numpy.max(y2)

        newCoor1 = []
        predict_score = 0
        sayac = 0

        for i in range(len(newCoor)):  
            predict_score += intersection_over_union(ses, newCoor[i])
            sayac+=1
            newCoor1.append(newCoor[i])
            
        #Birbiri ile ilişkili karelerin ortalaması alınarak son koordinatın çıkarılması
        if(newCoor1):
            predict_score = predict_score/sayac
            boxes = numpy.array(newCoor)
            x1 = boxes[:,0]
            y1 = boxes[:,1]
            x2 = boxes[:,2]
            y2 = boxes[:,3]
            predict = numpy.mean(x1),numpy.mean(y1),numpy.mean(x2),numpy.mean(y2)
            predict = numpy.array(predict).astype(int)
            
        if predict_score > 0.9: accuracy = "excellent"
        elif predict_score > 0.7: accuracy = "high"
        else: accuracy = "low"
        
        annatator.append({
                          "objName":"human",
                          "objCoordinates":predict,
                          "predict":predict_score,
                          "accuracy":accuracy
                         })
        
    print(annatator)
    image = cv2.imread('1.jpg')
    for i in range(len(annatator)):
        cv2.rectangle(image, annatator[i]["objCoordinates"][0:2], annatator[i]["objCoordinates"][2:4], (0, 255, 255), 2)
        cv2.putText(image, str(annatator[i]["predict"])[0:6] + " - " + str(annatator[i]["accuracy"]), (annatator[i]["objCoordinates"][0], annatator[i]["objCoordinates"][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 1)
    cv2.imshow("image", image)
    cv2.waitKey(0)
    
else:
    print('need three or more coordinates. .')

cv2.destroyAllWindows()